<a href="https://geo.nyu.edu/catalog/nyu_2451_34572?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ"><img src = "https://media12.s-nbcnews.com/i/MSNBC/Components/Video/202003/f_mash_usnscomfort_200330_1920x1080.jpg" align=center width = 1200> </a>

### Applied Data Science Capstone Final Project (The Battle of Neighborhoods - Week 2) 
   
### Zooming in: Finding the best location for a Hospital with a Development and Research Center in New York City after COVID-19 pandemic.
  
### Created by: Maria Hidalgo

### Introduction

<p style="text-align:justify"> COVID-19 is one of the worst pandemics the world has faced. There have been 80,155,187 confirmed cases, including 1,771,128 deaths, notified to the WHO at the end of 2020. These figures show how poorly prepared we are to face this type of problem. Many hospitals collapsed from the large number of patients arriving every hour, and some cities were hit harder than others. With this outlook came the idea of using the tools learned in this course <strong> to find the best location to build a Hospital that includes a development and research center in one of the most affected cities in the world during this pandemic, such as New York City.</strong> </p>


### Data and Methodology

<p style="text-align:justify"> The data to be used will be obtained from the following web address: https://geo.nyu.edu/catalog/nyu_2451_34572.
This data has a total of 5 boroughs and 306 neighborhoods from NYC. Also has the latitude and logitude coordinates of each neighborhood. </p>

<p style="text-align:justify"> I will convert addresses into their equivalent latitude and longitude values best fit to build the Hospital and the Development and Research Center (H&DRC). Also, I will use the Foursquare API to explore neighborhoods most affected by COVID-19 in New York City. I will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. I will use the _k_-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.</p>

### Problem Statement 

<p style="text-align:justify"> 
What is the best location to build the H&DRC in NYC based on post-pandemic needs? In what neighborhood should the autorities or organization open the H&DRC?
</p>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#import folium
#print( folium.__version__)

print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 1. Download and Explore Dataset

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web: [https://geo.nyu.edu/catalog/nyu_2451_34572](https://geo.nyu.edu/catalog/nyu_2451_34572?cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ)


In [2]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json
print('Data downloaded!')

Data downloaded!


In [3]:
##### Loading and exploring the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [4]:
#All the relevant data is in the _features_ key, which is basically a list of the neighborhoods. The variable that includes this data is:
neighborhoods_data = newyork_data['features']

In [5]:
#Tranforming the data into a _pandas_ dataframe
#Dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# The dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [6]:
#Filling the dataframe one row at a time.
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [7]:
#Making sure that the dataset has all 5 boroughs and 306 neighborhoods
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [8]:
#Using geopy library to get the latitude and longitude values of New York City.
#To define an instance of the geocoder, I defined a user_agent. The agent is named <em>ny_explorer</em>
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [9]:
# Map of New York with neighborhoods superimposed on top (using latitude and longitude values).

from folium import GeoJson
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.Circle(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)

          
map_newyork

In [10]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)

address1 = 'Brooklyn, NY'

geolocator1 = Nominatim(user_agent="ny_explorer")
location = geolocator1.geocode(address1)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude1, longitude1))

map_brooklyn = folium.Map(location=[latitude1, longitude1], zoom_start=11)
tooltip = 'Click me!'

for lat1, lng1, label1 in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label1 = folium.Popup(label1, parse_html=True)
    folium.CircleMarker(
        [lat1, lng1],
        radius=5,
        popup=label1,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
map_brooklyn
    
address2 = 'Queens, NY'

geolocator2 = Nominatim(user_agent="ny_explorer")
location = geolocator2.geocode(address2)
latitude2 = location.latitude
longitude2 = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude2, longitude2))

map_queens = folium.Map(location=[latitude2, longitude2], zoom_start=11)
#feature_group2 = folium.FeatureGroup(name='Brooklyn')

for lat2, lng2, label2 in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label2 = folium.Popup(label2, parse_html=True)
    folium.CircleMarker(
        [lat2, lng2],
        radius=5,
        popup=label2,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn) 
    folium.Marker([latitude2, longitude2], popup='<i>Queens</i>', icon=folium.Icon(color='green', icon="ok-sign")).add_to(map_brooklyn)
    folium.Marker([latitude1, longitude1], popup='<i>Brooklyn</i>',  icon=folium.Icon(color='red', icon="ok-sign")).add_to(map_brooklyn)
    
map_brooklyn

The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.
The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [11]:
#I am going to start using the Foursquare API to explore the neighborhoods and segment them. This is possible defining Foursquare Credentials and Version

CLIENT_ID = '5QA0VL4IFTFWSTUZTOIPST5BVDN3TMU4R3LQCZB1LX4AAQXV' # your Foursquare ID
CLIENT_SECRET = 'BMYFMJZ5L0JA1ZYIE41G1V4CAAZ44ZHYRRVEUVHJ2FJMNRHB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 5QA0VL4IFTFWSTUZTOIPST5BVDN3TMU4R3LQCZB1LX4AAQXV
CLIENT_SECRET:BMYFMJZ5L0JA1ZYIE41G1V4CAAZ44ZHYRRVEUVHJ2FJMNRHB


In [12]:
#Areas in Queens where there are not a Hospital
#queens_data.loc[29, 'Neighborhood']   #Fresh Meadows
#queens_data.loc[31, 'Neighborhood'] #Jamaica Center
#queens_data.loc[35, 'Neighborhood']  #South Jamaica
#queens_data.loc[33, 'Neighborhood']    #Queen Village
#queens_data.loc[36, 'Neighborhood']   # St. Albans
#queens_data.loc[3, 'Neighborhood'] Elmhurst

In [13]:
#Getting the neighborhood's latitude and longitude values.
neighborhood_latitude_queens = queens_data.loc[35, 'Latitude'] # neighborhood latitude value
neighborhood_longitude_queens = queens_data.loc[35, 'Longitude'] # neighborhood longitude value

neighborhood_name_queens = queens_data.loc[35, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name_queens, 
                                                               neighborhood_latitude_queens, 
                                                               neighborhood_longitude_queens))

Latitude and longitude values of South Jamaica are 40.696911253789885, -73.7904261313554.


In [14]:
#Now, let's get the top 100 venues that are in South Jamaica within a radius of 500 meters.
#First, let's create the GET request URL. Name your URL **url**.
# type your answeSout
search_query = 'South Jamaica'
radius = 500
print(search_query + ' .... OK!')

South Jamaica .... OK!


In [15]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, neighborhood_latitude_queens, neighborhood_longitude_queens, radius, LIMIT)
#url
results = requests.get(url).json()
#results
#We know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function.

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
#Now we are ready to clean the json and structure it into a _pandas_ dataframe.

In [17]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,The Nourish Spot,Vegetarian / Vegan Restaurant,40.697966,-73.792267
1,Anita's Roti Shop,Caribbean Restaurant,40.699471,-73.793022
2,Fine Fare Supermarkets,Supermarket,40.697310,-73.785833
3,Bravo Supermarkets,Grocery Store,40.695080,-73.790522
4,MTA Regional Bus Operations - Jamaica Depot,Bus Station,40.699847,-73.791718
5,MTA - Bus Stop,Bus Station,40.699835,-73.788708
6,Wonder Bread,Bakery,40.700571,-73.788868
7,J & C Deli & Grocery,Deli / Bodega,40.693180,-73.789124


In [18]:
#And how many venues were returned by Foursquare?
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


## 2. Exploring Neighborhoods in Queens


#### Let's create a function to repeat the same process to all the neighborhoods in Queens

In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
#### The above function is running on each neighborhood and it's created a new dataframe called _queens_venues_ that keep the venues in queens.

queens_venues=getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )


Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens


In [ ]:
print(queens_venues.shape)
queens_venues.head(10)

### How many venues were returned for each neighborhood?

In [ ]:
group_venues_queens=queens_venues.groupby('Neighborhood').count()
#group_venues_queens

### How many unique categories can be curated from all the returned venues?

In [ ]:
unique_categ_queens=print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))
unique_categ_queens

 ## 3. Analyzing Each Neighborhood

In [ ]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

In [ ]:
queens_onehot.shape

#### Next, grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
#queens_grouped

#### Printing each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Putting that into a _pandas_ dataframe

In [ ]:
#First, writing a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Second creating the new dataframe and display the top 5 venues for each neighborhood.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(50)

## 4. Cluster Neighborhoods

In [ ]:
#Run _k_-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [ ]:
#Creating a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge queens_grouped with queens_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#queens_merged.head() # check the last columns!

In [ ]:
queens_merged.head(50)

In [ ]:
#Adding queens hospitals data
data_hospitals_queens = pd.DataFrame({
'lat_h':[40.7466512,40.7165676,40.7678805,40.745155, 40.7016473, 40.7077414, 40.7165676, 40.7560705, 40.7150924,  40.5971296],
'lon_h':[-73.8255622,-73.804537,-73.9247404,-73.8856035, -73.8165716, -73.8117994, -73.804537, -73.8179433, -73.8308064, -73.7572871],
'Hospital':['Queens Hospital Center','NYC Medical Examiners Office of Queens', 'Mount Sinai Queens', 'Elmhurst Hospital: Center Ob/Gyn, Broadway, Queens, NY', 'Jamaica Hospital Medical Center', 'NewYork-Presbyterian/Queens Center for Wound Healing', 'Queens Cancer Center', 'Flushing Hospital Medical Center',  'New York Hospital Med Center-Queens', 'St. Johns Episcopal Hospital']
})
#data_hospitals_queens.head(10)


### Finally, visualizing the resulting clusters

In [ ]:
# create map
import matplotlib.pyplot as plt 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
for i in range(10):
    folium.Marker([data_hospitals_queens['lat_h'][i], data_hospitals_queens['lon_h'][i]], icon=folium.Icon(color='red',icon='medkit',prefix="fa"), popup=data_hospitals_queens['Hospital'][i]).add_to(map_clusters)
              
map_clusters


In [ ]:
pip install geopandas

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon

lat_point_list = [40.7027683,40.6699636,40.6596883,40.6938137,40.7242169]
lon_point_list = [-73.812999,-73.7986874, -73.7541751, -73.729493,-73.7544507]


polygon_geom = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")


folium.GeoJson(polygon).add_to(map_clusters)
folium.LatLngPopup().add_to(map_clusters)
map_clusters

## 5. Examining Clusters

### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

#### Cluster 0

In [ ]:
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

#### Cluster 1

In [ ]:
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

#### Cluster 2

In [ ]:
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

#### Cluster 3

In [ ]:
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

#### Cluster 4

In [ ]:
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

## 6. Exploring Neighborhoods in Brooklyn

#### The above function is running on each neighborhood and it's created a new dataframe called _queens_venues_ that keep the venues in Brooklyn.

In [ ]:
brooklyn_venues=getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

In [ ]:
print(brooklyn_venues.shape)
brooklyn_venues.head(10)

### How many venues were returned for each neighborhood?

In [ ]:
group_venues_brooklyn=brooklyn_venues.groupby('Neighborhood').count()
#group_venues_brooklyn

### How many unique categories can be curated from all the returned venues?

In [ ]:
unique_categ_brooklyn=print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))
unique_categ_brooklyn

 ## 3. Analyzing Each Neighborhood

In [ ]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

In [ ]:
brooklyn_onehot.shape

#### Next, grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
#brooklyn_grouped

#### Printing each neighborhood along with the top 5 most common venues

In [ ]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

### Putting that into a _pandas_ dataframe

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
#Creating the new dataframe and display the top 5 venues for each neighborhood.
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

## 7. Cluster Neighborhoods

In [ ]:
#Run _k_-means to cluster the neighborhood into 5 clusters.
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


In [ ]:
#Creating a new dataframe that includes the cluster as well as the top 5 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge queens_grouped with queens_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [ ]:
brooklyn_merged.head(10)

In [ ]:
#Adding brooklyn hospitals data
data_hospitals_brooklyn = pd.DataFrame({
'lat_h':[40.6253052,40.6391639,40.6463875,40.6190874,40.614041,40.5856299,40.6551152,40.6559927, 40.667636,40.6906959,40.6996727,40.7036562],
'lon_h':[-73.9908354,-73.9982025, -74.0220039,-73.9430475,-73.9486354,-73.9651935,-73.912439,-73.9448033,-73.9787852,-73.9771816,-73.9424001,-73.9175794],
'Hospital':['NewYork-Presbyterian Medical Group Brooklyn Park Slope', 'Maimonides Medical Center Weinberg Emergency Center', 'NYU Langone Hospital Brooklyn','Mount Sinai Brooklyn', 'New York Community Hospital','NYC Health + Hospitals/Coney Island', 'Brookdale University Hospital Medical Center', 'University Hospital-Brooklyn', 'New York Methodist Hospital Ob/ Gyn', 'Brooklyn Hospital', 'NYC Health + Hospitals/Woodhull', ' Wyckoff Heights Medical Center']
})
#data_hospitals_brooklyn.head(20)


### Finally, visualizing the resulting clusters

In [ ]:
# create map
map_clusters_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_brooklyn)
for i in range(11):
    folium.Marker([data_hospitals_brooklyn['lat_h'][i], data_hospitals_brooklyn['lon_h'][i]], icon=folium.Icon(color='red',icon='medkit',prefix="fa"), popup=data_hospitals_brooklyn['Hospital'][i]).add_to(map_clusters_brooklyn)
              
map_clusters_brooklyn

In [ ]:

lat_point_list_br = [40.6304371,40.6147166,40.584891,40.612577]
lon_point_list_br = [-74.0110586,-74.0241213,-73.9841167,-73.9547995]

polygon_geom = Polygon(zip(lon_point_list_br, lat_point_list_br))
crs = {'init': 'epsg:4326'}
polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
print(polygon.geometry)

polygon.to_file(filename='polygon.geojson', driver='GeoJSON')
polygon.to_file(filename='polygon.shp', driver="ESRI Shapefile")


folium.GeoJson(polygon).add_to(map_clusters_brooklyn)
folium.LatLngPopup().add_to(map_clusters_brooklyn)
map_clusters_brooklyn

## 8. Examining Clusters

#### Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

### Cluster 0


In [ ]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

### Cluster 1

In [ ]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

### Cluster 2

In [ ]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

### Cluster 3

In [ ]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

### Cluster 4

In [ ]:
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]